In [1]:
#tutorial: https://www.tensorflow.org/tutorials/estimators/cnn
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Imports
import numpy as np
import tensorflow as tf
from numpy import genfromtxt

tf.logging.set_verbosity(tf.logging.INFO)

def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    # Input Layer
    # [batch_size, width, height, number of channels]
    # set batch to -1 to be tunneable
    input_layer = tf.reshape(features["x"], [-1, 100, 100, 1])

    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5], #dimensions of the filters [width, height]
      padding="same", #input tensor will have the same dimensions as output tensor
      activation=tf.nn.relu) #activation function Re(ctified) L(inear) (U)nit

    # Pooling Layer #1, decreases dimensionality and hence processing time
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
    
    # Dense Layer
    pool2_flat = tf.reshape(pool2, [-1, 10 * 100 * 40])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)

    dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)


    # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=4)

    predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


In [6]:
def main(unused_argv):
  # Load training and eval data
    stefanie = genfromtxt('stefanie.csv', delimiter=',')
    jimmy = genfromtxt('jonathan_jimmy.csv', delimiter=',')
    harry = genfromtxt('harry_marquez.csv', delimiter=',')
    delaGala = genfromtxt('delaGala.csv', delimiter=',')
    train_labels = []
    eval_labels = []
    # begin processing stefanie csv
    temp_train = stefanie[:90]
    temp_eval = stefanie[91:]
    train_data = temp_train 
    eval_data = temp_eval 
    for i in temp_train:
        train_labels.append(1)
    for i in temp_eval:
        eval_labels.append(1)
    #end processing stefanie

    #begin processing jimmy csv
    temp_train = jimmy[:90]
    temp_eval = jimmy[91:]
    for i in temp_train:
        train_labels.append(0)
    for i in temp_eval:
        eval_labels.append(0)
    train_data = np.vstack((train_data, temp_train))
    eval_data = np.vstack((eval_data, temp_eval))
    #end processing jimmy csv

    #begin processing harry
    temp_train = harry[:90]
    temp_eval = harry[91:]
    for i in temp_train:
        train_labels.append(3)
    for i in temp_eval:
        eval_labels.append(3)
    train_data = np.vstack((train_data, temp_train))
    eval_data = np.vstack((eval_data, temp_eval))
    #end processing harry

    #begin processing delaGala
    temp_train = delaGala[:90]
    temp_eval = delaGala[91:]
    for i in temp_train:
        train_labels.append(2)
    for i in temp_eval:
        eval_labels.append(2)
    train_data = np.vstack((train_data, temp_train))
    eval_data = np.vstack((eval_data, temp_eval))
    #end processing delaGala
    eval_labels = np.array(eval_labels)

    train_labels = np.array(train_labels)
    train_data = train_data.astype(np.float32)
    eval_data = eval_data.astype(np.float32)
    eval_labels = eval_labels.astype(np.int32)
    train_labels = train_labels.astype(np.int32)
    # Create the Estimator
    mnist_classifier = tf.estimator.Estimator(
        model_fn=cnn_model_fn)
    # Set up logging for predictions
    tensors_to_log = {"probabilities": "softmax_tensor"}
    logging_hook = tf.train.LoggingTensorHook(
        tensors=tensors_to_log, every_n_iter=50)
    # Train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": train_data},
        y=train_labels,
        batch_size=90,
        num_epochs=None,
        shuffle=True)
    #mnist_classifier.train(
    #    input_fn=train_input_fn,
    #    steps=500,
    #    hooks=[logging_hook])
    #Evaluate the model and print results
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": eval_data},
        y=eval_labels,
        num_epochs=1,
        shuffle=False)
    eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
    print(eval_results)



In [7]:
if __name__ == "__main__":
    tf.app.run()

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/_h/_ygnszg119j4lpn4cqwsdwwc0000gn/T/tmpey5mf05z', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0xb2a441278>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Could not find trained model in model_dir: /var/folders/_h/_ygnszg119j4lpn4cqwsdwwc0000gn/T/tmpey5mf05z, running initialization to evaluate.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-12-05

SystemExit: 